In [25]:
%%capture --no-stderr
%pip install --upgrade --quiet langchain langchain-community langchain-openai faiss-cpu

In [26]:
import getpass
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI

os.environ["LANGCHAIN_TRACING_V2"] = "true" 

load_dotenv()
if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")
if "LANGCHAIN_API_KEY" not in os.environ:
    os.environ["LANGCHAIN_API_KEY"] = getpass.getpass("Enter your LangChain API key: ")

llm = ChatOpenAI(model="gpt-4o-mini")

In [27]:
from langchain_community.utilities import SQLDatabase

db = SQLDatabase.from_uri("sqlite:///Chinook.db") # select relative database file

# print(db.dialect)
# print(db.get_usable_table_names())
# db.run("SELECT * FROM Artist LIMIT 10;")

In [28]:
from typing_extensions import TypedDict

#schema for the final dictionary and pipeline
class State(TypedDict):
    question: str
    query: str
    result: str
    answer: str

In [29]:
from langchain_core.prompts import ChatPromptTemplate

system_message = """
Given an input question, create a syntactically correct {dialect} query to
run to help find the answer. Unless the user specifies in his question a
specific number of examples they wish to obtain, always limit your query to
at most {top_k} results. You can order the results by a relevant column to
return the most interesting examples in the database.

Never query for all the columns from a specific table, only ask for a the
few relevant columns given the question.

Pay attention to use only the column names that you can see in the schema
description. Be careful to not query for columns that do not exist. Also,
pay attention to which column is in which table.

Only use the following tables:
{table_info}
"""

user_prompt = "Question: {input}" 

query_prompt_template = ChatPromptTemplate(
    [("system", system_message),
    ("human", user_prompt)],
)

for message in query_prompt_template.messages:
    message.pretty_print()

================================ System Message ================================


Given an input question, create a syntactically correct {dialect} query to
run to help find the answer. Unless the user specifies in his question a
specific number of examples they wish to obtain, always limit your query to
at most {top_k} results. You can order the results by a relevant column to
return the most interesting examples in the database.

Never query for all the columns from a specific table, only ask for a the
few relevant columns given the question.

Pay attention to use only the column names that you can see in the schema
description. Be careful to not query for columns that do not exist. Also,
pay attention to which column is in which table.

Only use the following tables:
{table_info}

================================ Human Message =================================

Question: {input}


In [30]:
# convert prompt to query using llm itself
from typing_extensions import Annotated

class QueryState(TypedDict):
    query: Annotated[str, ..., "A syntactically correct SQL query to run."] 
    # define the structure of the llm output to be a dictionary with a query string (so we dont get extra text like "The query is: ...")

# populate parameters for the query prompt
def write_query(state: State):
    # Fill in the template with a dict, not kwargs
    prompt = query_prompt_template.invoke({
        "dialect": db.dialect,
        "top_k": 10,
        "table_info": db.get_table_info(),
        "input": state["question"],
    })
    structured_llm = llm.with_structured_output(QueryState)  # use the llm with structured output
    result = structured_llm.invoke(prompt)
    return result

write_query({"question": "How many tracks are there in the database?"})

{'query': 'SELECT COUNT(*) AS TrackCount FROM Track;'}

In [ ]:
# 2. execute the query: excexute the generated sql query
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool

def execute_query(state: State):
    execute_query_tool = QuerySQLDataBaseTool(db=db)            
    return {"result": execute_query_tool.invoke(state["query"])} # use the QuerySQLDataBaseTool to execute the query

execute_query({'query': 'SELECT COUNT(*) as TotalTracks FROM Track;'})

{'result': '[(3503,)]'}

In [38]:
# 3. generate answer from the result
def generate_answer(state: State):
    prompt = (
        "Given the following user question, corresponding SQL query, "
        "and SQL result, answer the user question.\n\n"
        f'Question: {state["question"]}\n'
        f'SQL Query: {state["query"]}\n'
        f'SQL Result: {state["result"]}'
    )
    response = llm.invoke(prompt)
    return {"answer": response.content}

In [ ]:
from langgraph.graph import START, StateGraph
# build the pipeline using LangGraph
graph_builder = StateGraph(State).add_sequence(
    [write_query, execute_query, generate_answer]
)
# add entry point to tell graph where to start
graph_builder.add_edge(START, "write_query")
graph = graph_builder.compile()

In [ ]:
from IPython.display import Image, display
from langchain_core.runnables.graph import CurveStyle, MermaidDrawMethod, NodeStyles

print(graph.get_graph().draw_mermaid()) # text representation of the graph
try:
    with open("sql_graph.mmd", "w") as f:
        f.write(graph.get_graph().draw_mermaid())  
        # i cant display the graph in this notebook, so saving it to a file
        # mmdc -i sql_graph.mmd -o sql_graph.png
except Exception:
    pass

---
config:
  flowchart:
    curve: linear
---
graph TD;
	__start__([<p>__start__</p>]):::first
	write_query(write_query)
	execute_query(execute_query)
	generate_answer(generate_answer)
	__end__([<p>__end__</p>]):::last
	__start__ --> write_query;
	execute_query --> generate_answer;
	write_query --> execute_query;
	generate_answer --> __end__;
	classDef default fill:#f2f0ff,line-height:1.2
	classDef first fill-opacity:0
	classDef last fill:#bfb6fc



In [52]:
for step in graph.stream(
    {"question": "How many employees are there?"}, stream_mode="updates"
):
    print(step)


{'write_query': {'query': 'SELECT COUNT(*) AS EmployeeCount FROM Employee;'}}
{'execute_query': {'result': '[(8,)]'}}
{'generate_answer': {'answer': 'There are 8 employees.'}}
